<a href="https://colab.research.google.com/github/profteachkids/CHE2064_Spring2023/blob/main/CHE2064Regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [23]:
import numpy as np 
from plotly.subplots import make_subplots
from scipy.optimize import minimize

In [6]:
x= np.linspace(0,5, 25)

In [7]:
ydata = 0.5*(x**2) + 2.*x + 1.
ydata = np.random.normal(ydata, scale=0.3, )

In [ ]:
fig = make_subplots()
fig.add_scatter(x=x,y=ydata,mode='markers')

In [9]:
m,b=np.polyfit(x,ydata,1)

In [ ]:
fig.add_scatter(x=x,y=m*x+b,mode='lines')

In [ ]:
a,b,c = np.polyfit(x,ydata,2)
fig.add_scatter(x=x,y=a*x**2+b*x+c,mode='lines')

In [13]:
a,b,c

(0.5388180534053119, 1.7997717125499149, 1.1026449750486562)

In [20]:
A=np.c_[x**2, x, np.ones_like(x)]

In [18]:
A.T @ A

array([[3321.17904851,  813.80208333,  212.67361111],
       [ 813.80208333,  212.67361111,   62.5       ],
       [ 212.67361111,   62.5       ,   25.        ]])

In [22]:
np.linalg.solve(A.T @ A, A.T @ ydata)

array([0.53881805, 1.79977171, 1.10264498])

In [26]:
def sse(v):
    a,b,c = v
    model = a*x**2 + b*x + c
    return np.sum((ydata-model)**2)

In [28]:
minimize(sse, (0., -1000., 0.5))

      fun: 1.7899683327401876
 hess_inv: array([[ 0.00493172, -0.02465842,  0.01969195],
       [-0.02465842,  0.13215287, -0.12061361],
       [ 0.01969195, -0.12061361,  0.15401651]])
      jac: array([1.34110451e-07, 1.19209290e-07, 4.47034836e-08])
  message: 'Optimization terminated successfully.'
     nfev: 56
      nit: 8
     njev: 14
   status: 0
  success: True
        x: array([0.53881788, 1.79977255, 1.10264434])

In [29]:
!wget https://raw.githubusercontent.com/profteachkids/CHE2064_Spring2023/main/WaterVaporPressure.csv

--2023-01-26 18:10:01--  https://raw.githubusercontent.com/profteachkids/CHE2064_Spring2023/main/WaterVaporPressure.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1300 (1.3K) [text/plain]
Saving to: ‘WaterVaporPressure.csv’

WaterVaporPressure. 100%[===================>]   1.27K  --.-KB/s    in 0s      

2023-01-26 18:10:01 (97.3 MB/s) - ‘WaterVaporPressure.csv’ saved [1300/1300]



In [32]:
PvapData=np.loadtxt('WaterVaporPressure.csv',delimiter=',')

In [43]:
T, Pexp = PvapData.T
logPexp = np.log10(Pexp)

In [46]:
np.polyfit(1/T, logPexp, 1)

array([-2113.94814059,    10.63549233])

In [40]:
def antoine_sse(v):
    a,b,c = v
    model = 10**(a-b/(T+c))
    return np.sum((model - Pexp)**2)

In [48]:
def antoinelog10_sse(v):
    a,b,c = v
    model = (a-b/(T+c))
    return np.sum((model - logPexp)**2)

In [61]:
np.log10(99000)

4.99563519459755

In [51]:
antoine_abc=minimize(antoine_sse,(10.21,1756.,-36.)).x

In [52]:
antoine_abc

array([  10.5016998 , 2064.92654514,    5.87206213])

In [53]:
antoinelog10_abc=minimize(antoinelog10_sse,(10.64,2114.,0.)).x

In [56]:
def antoine(v):
    a,b,c=v
    return 10**(a-b/(T+c))

In [60]:
fig2=make_subplots()
fig2.add_scatter(x=T, y=Pexp, mode='markers')
fig2.add_scatter(x=T, y=antoine(antoine_abc), mode='lines',name='Pexp')
fig2.add_scatter(x=T, y=antoine(antoinelog10_abc), mode='lines',name='logPexp')
fig2.update_yaxes(type="log")